# YOLO11 Detection Config (yolo11.yaml)
文件：`ultralytics/cfg/models/11/yolo11.yaml`
用途：YOLO11 检测模型定义，输出 P3/8、P4/16、P5/32 三个尺度的检测头。


## 参数与模型族
- `nc`: 80 类（COCO 默认，可自定义）。
- `scales`: 通过不同深度、宽度、`max_channels` 复用同一骨干/头定义：
  - `n`: depth=0.50, width=0.25, max_channels=1024（轻量）
  - `s`: depth=0.50, width=0.50, max_channels=1024
  - `m`: depth=0.50, width=1.00, max_channels=512
  - `l`: depth=1.00, width=1.00, max_channels=512
  - `x`: depth=1.00, width=1.50, max_channels=512
- 通过 `model=yolo11n.yaml` 等别名调用时会选择对应缩放系数。


## Backbone（特征提取，逐行解释）
- YAML 行格式 `[from, repeats, module, args]`：`from=-1` 表示上一层输出，`repeats` 受 depth 系数缩放，`module(args)` 为要堆叠的层。
- 输入 → 输出特征金字塔：P1/2 → P2/4 → P3/8 → P4/16 → P5/32。
- 阶段详解（括号为参数含义）：
  1. `Conv(64, k3, s2)`：首个卷积，下采样 2x，输出 64 通道。
  2. `Conv(128, k3, s2)` → `C3k2(256, repeat=2, shortcut=False, e=0.25)`：下采样到 P2/4；CSP 风格块，`e` 控制 bottleneck 压缩比，`shortcut=False` 关闭残差。
  3. `Conv(256, k3, s2)` → `C3k2(512, repeat=2, shortcut=False, e=0.25)`：下采样到 P3/8（对小目标有用）。
  4. `Conv(512, k3, s2)` → `C3k2(512, repeat=2, shortcut=True)`：下采样到 P4/16，保持 512 通道，开启残差更易训练。
  5. `Conv(1024, k3, s2)` → `C3k2(1024, repeat=2, shortcut=True)`：下采样到 P5/32，高层语义特征。
  6. `SPPF(1024, k5)`：多尺度池化，扩大感受野，保持分辨率不变。
  7. `C2PSA(1024, repeat=2)`：部分自注意力，引入长程依赖，强化高层语义。
- 好处：逐级下采样+瓶颈降低计算，SPPF 提升全局上下文，C2PSA 让高层特征具备更强语义表达。


## Neck + Head（特征融合与检测，标出 Neck 模块）
- Neck = FPN/PAN 融合（上采样/下采样 + Concat + C3k2），Head = Detect。
- Neck 具体步骤：
  1. **(Neck)** `Upsample(scale=2)` 将 P5 放大 → 与 P4 `Concat(dim=1)` → `C3k2(512, repeat=2, shortcut=False)`，得到融合后的 P4。
  2. **(Neck)** 再 `Upsample(scale=2)` 将 P4 放大 → 与 P3 `Concat` → `C3k2(256, repeat=2, shortcut=False)`，得到 P3（专注小目标）。
  3. **(Neck)** `Conv(256, k3, s2)` 下采样 P3 → 与上一步的 P4 `Concat` → `C3k2(512, repeat=2, shortcut=False)`，得到新的 P4（中目标）。
  4. **(Neck)** `Conv(512, k3, s2)` 下采样 P4 → 与之前的 P5 `Concat` → `C3k2(1024, repeat=2, shortcut=True)`，得到新的 P5（大目标）。
- Head：
  5. **(Head)** `Detect([P3, P4, P5], nc)`：三尺度检测头，同时预测类别与框。
- 设计好处：
  - 自顶向下（上采样）把高层语义带到低层，帮助小目标。
  - 自底向上（下采样）补充细节到高层，强化中/大目标。
  - 三尺度输出覆盖小/中/大目标，平衡精度与速度。


## 模块与参数小词典（通俗版）
- `Conv(out_c, k, s)`: 卷积+BN+激活；`k`=卷积核大小，`s`=步幅，`s>1` 会下采样。
- `C3k2(out_c, shortcut, e)`: CSP 风格块，拆分通道后堆叠 3 个 bottleneck；`e` 控制 bottleneck 压缩比例；`shortcut` 是否开残差。
- `SPPF(out_c, k)`: 快速空间金字塔池化，多次最大池化叠加，扩大感受野，计算量小。
- `C2PSA(out_c)`: C2 Block + Partial Self-Attention，加入部分自注意力，增强全局信息。
- `nn.Upsample(scale=2, mode="nearest")`: 上采样，放大特征图尺寸。
- `Concat(dim=1)`: 按通道拼接两个特征，实现特征融合。
- `Detect([P3,P4,P5], nc)`: 多尺度检测头，输出每个尺度的类别与边框参数。
- `from` 索引：指向前面某层输出（`-1`=上一层）；列表如 `[-1,6]` 取两层输出后拼接。
- `repeats`: 会乘以 depth multiplier，决定堆叠次数；本 yolo11 n/s/m 深度系数 0.5，层数一致。
- 通道/宽度缩放：`width multiplier` 缩放各层通道；`max_channels` 限制放大后的上限，防止高层通道过宽。


## 信息流顺序（一句话版）
- 输入图像 → Backbone 连续下采样得到 P3/P4/P5。
- Head：先上采样 P5→P4、P4→P3 带语义；再下采样 P3→P4、P4→P5 补细节；最终 Detect 输出三尺度框/类别。
- FPN + PAN 的组合，让低层细节和高层语义互补，适配不同尺寸目标。


## 代码路径提示
- 配置：`ultralytics/cfg/models/11/yolo11.yaml`（当前文件）。
- 构建逻辑：`ultralytics/models/yolo/model.py` 读取 YAML，调用 `ultralytics/nn/modules/` 中的 `Conv`、`C3k2`、`SPPF`、`C2PSA` 等模块。
- 检测头：`Detect` 定义在 `ultralytics/nn/modules/head.py`（由 YAML 解析时动态引用）。
- 训练/推理入口：`ultralytics/engine/model.py`（封装 train/val/predict/export）。
